# Question 3: CSP

### TWO + TWO = FOUR

Sources:

https://ktiml.mff.cuni.cz/~bartak/constraints/consistent.html

https://ics.uci.edu/~dechter/courses/ics-276/spring-19/reading/chapter3-constraints.pdf

In [3]:
arcDebug = True
pathDebug = True
vars = ['T', 'W', 'O', 'F', 'U', 'R']
domain = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# Need to make sure im not using references!!!!
D = [domain.copy(),
     domain.copy(),
     domain.copy(),
     domain.copy(),
     domain.copy(),
     domain.copy()]
# Manually solve node consistency
D[3] = [1]
D[0] = [5, 6, 7, 8, 9]
# Defining  constraints
# Digonals:
def CDi(x, y):
    return True
# Default:
def CDe(x, y):
    return x != y
# C00: diagonal
# C01: default
def C02(T, O):
    return (T != O) and ((O == (2*T)%10) or O == ((2*T)%10 + 1))
# C03: default
# C04: default
# C05: default
# C10: default
# C11: diagonal
def C12(W, O):
    return (O != W) and ((O%2 == 1) == ((2*W)/10 >= 1))
# C13: default
def C14(W, U):
    return (W != U) and ((U == (2*W)%10) or (U == (2*W)%10 + 1))
# C15: default
def C20(O, T):
    return (T != O) and ((O == (2*T)%10) or (O == (2*T)%10 + 1))
def C21(O, W):
    return (O != W) and ((O%2 == 1) == ((2*W)/10 >= 1))
# C22: diagonal
# C23: default
def C24(O, U):
    return (O != U) and ((U%2 == 1) == ((2*O)/10 >= 1))
def C25(O, R):
    return (O != R) and (R == (2*O)%10)
# C30: default
# C31: default
# C32: default
# C33: diagonal
# C34: default
# C35: default
# C40: default
def C41(U, W):
    return (W != U) and (U == (2*W)%10 or U == (2*W)%10 + 1)
def C42(U, O):
    return (O != U) and ((U%2 == 1) == ((2*O)/10 >= 1))
# C43: default
# C44: diagonal
# C45: default
def C52(R, O):
    return (O != R) and (R == (2*O)%10)
# rest are defaults or diagonals

# Consistency matrix
C = [[CDi, CDe, C02, CDe, CDe, CDe],
     [CDe, CDi, C12, CDe, C14, CDe],
     [C20, C21, CDi, CDe, C24, C25],
     [CDe, CDe, CDe, CDi, CDe, CDe],
     [CDe, C41, C42, CDe, CDi, CDe],
     [CDe, CDe, C52, CDe, CDe, CDi]]


# For the arc Aij, checks that each value in Di has a value in Dj
# that satisfies Cij
def Revise(i, j):
    idxsToRemove = []
    idx = 0
    for v1 in D[i]:
        found = False
        for v2 in D[j]:
            if C[i][j](v1, v2):
                found = True
                break
        if not found:
            if arcDebug:
                print("Removing:", vars[i], "=", v1)
                print("\t ", vars[j], ':', D[j])
            idxsToRemove.append(idx)
        idx += 1
    for idx in reversed(idxsToRemove): # Need to go in reverse to delete correctly
        del D[i][idx]
    return len(idxsToRemove) > 0

# For each two variables, check that they are arc consistent and repeat until
# there are no inconsistencies.
def AC1(): 
    revised = True
    while revised:
        revised = False
        for i in range(6):
            for j in range(6):
                revised = revised or Revise(i, j)

# Really not sure if this is right, but it eliminates some extra
# elements of the domains, and gives the same final result as just
# using arc consistency or brute force
# Check that for any three varibales i, j, k, there are vi, vj, vk in
# Di, Dj, Dk, that all satisfy eachother's constraints.
def Revise3(i, j, k):
    idxsToRemove = []
    idx = 0
    for Vi in D[i]:
        found = False
        for Vj in D[j]:
            for Vk in D[k]:
                # Check for consistency between all three values
                found = found or (C[k][j](Vk, Vj) and C[i][j](Vi, Vj) and C[i][k](Vi, Vk))
        if not found:
            if pathDebug:
                print("Removing:", vars[i], "=", Vi)
                print("\t ", vars[j], ':', D[j])
                print("\t ", vars[k], ':', D[k])
            idxsToRemove.append(idx)
        idx += 1
    for idx in reversed(idxsToRemove): # Need to go in reverse to delete correctly
        del D[i][idx]
    return len(idxsToRemove) > 0
            
    isToRemoveUnique = list(set(isToRemove))
    ksToRemoveUnique = list(set(ksToRemove))

    for idx in reversed(isToRemoveUnique):
        del D[i][idx]
    for idx in reversed(ksToRemoveUnique):
        print(D[k])
        print(idx)
        del D[k][idx]
    return (len(isToRemove) > 0) or (len(ksToRemove) > 0)

# Check path consistency for every combination of three variables
# until everything is path consistent
def PC(): 
    revised = True
    while revised:
        revised = False
        for i in range(6):
            for j in range(6):
                for k in range(6):
                    revised = revised or Revise3(i, j, k)
print("Node Consistent:")
for i in range(6):
    print(f"{vars[i]}: {D[i]}")
print()
AC1()
print("\nArc Consistent:")
for i in range(6):
    print(f"{vars[i]}: {D[i]}")
print()
PC()
print("\nPath Consistent:")
for i in range(6):
    print(f"{vars[i]}: {D[i]}")

# Now we have to find the paths.
# I'm using DFS which might defeat the point of this exercize, but I have no clue.
paths = []
def getPaths(currentPath):
    if len(currentPath) == 0:
        for i in range(len(D[0])):
            currentPath.append(D[0][i])
            getPaths(currentPath)
            del currentPath[-1]
    else:
        # For every value in the current variable's domain,
        # check that it is consistent with all other nodes in the path
        varIdx = len(currentPath)
        currentD = D[varIdx]
        
        for valIdx in range(len(currentD)):
            # for every element of the current path, check consistency with new node
            consistent = True
            for pathIdx in range(len(currentPath)):
                if not C[varIdx][pathIdx](currentD[valIdx], currentPath[pathIdx]):
                    consistent = False
            if consistent: 
                currentPath.append(currentD[valIdx])
                if len(currentPath) == 6:
                    paths.append(currentPath.copy())
                else:
                    getPaths(currentPath)
                del currentPath[-1]
getPaths([])
# Paths now contains all solutions
print("\nSolutions:")
for path in paths:
    print(path)
print()
# Print solutions in correct format
for path in paths:
    print(f"{path[0]}{path[1]}{path[2]}", "+", f"{path[0]}{path[1]}{path[2]}", "=", f"{path[3]}{path[2]}{path[4]}{path[5]}")

Node Consistent:
T: [5, 6, 7, 8, 9]
W: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
O: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
F: [1]
U: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
R: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Removing: W = 1
	  F : [1]
Removing: O = 9
	  T : [5, 6, 7, 8, 9]
Removing: O = 1
	  F : [1]
Removing: O = 0
	  R : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Removing: T = 5
	  O : [2, 3, 4, 5, 6, 7, 8]
Removing: U = 1
	  F : [1]
Removing: W = 0
	  U : [0, 2, 3, 4, 5, 6, 7, 8, 9]
Removing: R = 1
	  O : [2, 3, 4, 5, 6, 7, 8]
Removing: R = 3
	  O : [2, 3, 4, 5, 6, 7, 8]
Removing: R = 5
	  O : [2, 3, 4, 5, 6, 7, 8]
Removing: R = 7
	  O : [2, 3, 4, 5, 6, 7, 8]
Removing: R = 9
	  O : [2, 3, 4, 5, 6, 7, 8]

Arc Consistent:
T: [6, 7, 8, 9]
W: [2, 3, 4, 5, 6, 7, 8, 9]
O: [2, 3, 4, 5, 6, 7, 8]
F: [1]
U: [0, 2, 3, 4, 5, 6, 7, 8, 9]
R: [0, 2, 4, 6, 8]

Removing: O = 3
	  T : [6, 7, 8, 9]
	  R : [0, 2, 4, 6, 8]
Removing: W = 7
	  T : [6, 7, 8, 9]
	  O : [2, 4, 5, 6, 7, 8]
Removing: W = 5
	  O : [2, 4, 5, 6, 7, 8]
	  U : [0, 2, 3, 4